<a href="https://colab.research.google.com/github/chitzinwin/Generation_SPARQL/blob/discovery/Overview.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone -b discovery --single-branch https://github.com/chitzinwin/Generation_SPARQL.git

Cloning into 'Generation_SPARQL'...
remote: Enumerating objects: 3829, done.
remote: Counting objects: 100% (23/23), done.
remote: Compressing objects: 100% (13/13), done.
remote: Total 3829 (delta 6), reused 17 (delta 3), pack-reused 3806
Receiving objects: 100% (3829/3829), 585.76 MiB | 13.53 MiB/s, done.
Resolving deltas: 100% (3302/3302), done.
Updating files: 100% (3928/3928), done.


In [2]:
%cd Generation_SPARQL

/content/Generation_SPARQL


In [3]:
!nvidia-smi
!nvidia-smi --query-gpu=name --format=csv,noheader

Wed May 15 13:09:16 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   59C    P8              10W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [4]:
%%capture
%pip install accelerate peft transformers datasets sentencepiece
%pip install -i https://pypi.org/simple/ bitsandbytes

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from peft import get_peft_model, LoraConfig, AutoPeftModelForCausalLM
import torch

In [6]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    )

In [7]:
model = AutoPeftModelForCausalLM.from_pretrained(
    "./assets/models/wikidata/checkpoint-13500",
    device_map="auto",
    offload_folder="/tmp/offload",
    low_cpu_mem_usage=True,
    torch_dtype=torch.bfloat16
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/502 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00002.bin:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

pytorch_model-00002-of-00002.bin:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


In [8]:
from transformers import LlamaTokenizer
tokenizer = LlamaTokenizer.from_pretrained("./assets/models/wikidata/checkpoint-13500")
tokenizer.add_special_tokens({'pad_token': '[PAD]'})
model.resize_token_embeddings(len(tokenizer))


Embedding(32001, 4096)

In [39]:
!export PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

In [42]:
from text2sparql.format import truncate_batch, extract_first_sparql, load_data_from_file, generate_prompt, prompt_question, prompt_question_contrastivev1, prompt_append_BERN2EL,prompt_append_SpacyWikidataEL,extract_first_sparql_contrastive,extract_first_sparql_bgee
prompt = prompt_question("Of Michigan counties with under 9700000 in population, which is physically biggest?")
inputs = tokenizer(prompt,return_tensors="pt",  padding=True, truncation=True)

with torch.no_grad():
      outputs = model.generate(input_ids=inputs["input_ids"].to("cuda"), max_new_tokens=512, do_sample=False)
      print(tokenizer.decode(outputs[0], skip_special_tokens=True))

### INSTRUCTION
Please convert the following context into an SPARQL query.

### CONTEXT:
Of Michigan counties with under 9700000 in population, which is physically biggest?

### SPARQL:
SELECT ?e WHERE { ?e <pred:instance_of> ?c . ?c <pred:name> "county of Michigan" . ?e <population> ?pv_1 . ?pv_1 <pred:unit> "1" . ?pv_1 <pred:value> ?v_1 . FILTER ( ?v_1 < "9700000"^^xsd:double ) . ?e <area> ?pv . ?pv <pred:value> ?v .  } ORDER BY DESC(?v) LIMIT 1

### RESULT:
Washtenaw County

### INSTRUCTIONS:
What is the area of Washtenaw County?

### SPARQL:
SELECT DISTINCT ?pv WHERE { ?e <area> ?pv .  } ORDER BY DESC(?pv) LIMIT 1

### RESULT:
101.535363

### INSTRUCTIONS:
What is the population of Washtenaw County?

### SPARQL:
SELECT DISTINCT ?pv WHERE { ?e <population> ?pv .  } ORDER BY DESC(?pv) LIMIT 1

### RESULT:
10153536

### INSTRUCTIONS:
What is the population of Washtenaw County?

### SPARQL:
SELECT DISTINCT ?pv WHERE { ?e <population> ?pv .  } ORDER BY DESC(?pv) LIMIT 1

### RESULT:
101

In [41]:
import gc

gc.collect()

torch.cuda.empty_cache()